# Fine-tune Gemma 2b it in Google Colab


In [1]:
# Install virtualenv package
!pip install -q virtualenv

# Create a new virtual environment
!virtualenv finetuninggemmabase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 23.6 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 1155ms
  creator CPython3Posix(dest=/content/finetuninggemmabase, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
# Upgrade and install required libraries
# Python 3.10.12
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.2
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1
!pip3 install -q -U huggingface_hub
!pip3 install -q -U py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
# Activate the virtual environment
!source finetuninggemmabase/bin/activate

In [4]:
import os
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import huggingface_hub
from google.colab import userdata

In [5]:
def clear_memory():
    """Clear GPU memory and collect garbage."""
    torch.cuda.empty_cache()
    gc.collect()

# Clear memory before starting
clear_memory()

In [6]:
# Retrieve and set up Hugging Face token
hf_token = userdata.get("HF_TOKEN")
os.environ["WANDB_DISABLED"] = "true"

In [7]:
MODEL_ID = "google/gemma-2b-it"
REPO_NAME = "nguyenanhhuy248/gemma-2b-it-samsum"

In [8]:
# Load and configure the model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},
    token=hf_token
)
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [11]:
# Load and preprocess the dataset
dataset = load_dataset("samsum", split="train", trust_remote_code=True)
dataset = dataset.map(lambda samples: tokenizer(samples["dialogue"]), batched=True)

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask'],
    num_rows: 14732
})

In [16]:
# Create LoRA configuration for the model
lora_config = LoraConfig(
    r=8,
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

In [ ]:
# Function to format examples for training
def formatting_func(example):
    text = f"Dialogue: {example['dialogue'][0]}\nSummary: {example['summary'][0]}"
    return [text]

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=2048,
)

In [22]:
# Train the model
trainer.train()

Step,Training Loss
1,3.976500
2,4.421100
3,3.993200
4,4.188400
5,3.419600
6,3.000500
7,3.560500
8,3.001900
9,2.731400
10,2.788000


TrainOutput(global_step=100, training_loss=0.8752402050793171, metrics={'train_runtime': 182.5831, 'train_samples_per_second': 2.191, 'train_steps_per_second': 0.548, 'total_flos': 1054504380948480.0, 'train_loss': 0.8752402050793171, 'epoch': 26.67})

In [23]:
# Save trained model
trainer.model.save_pretrained("gemma-2b-it-samsum")

In [24]:
# Clean up
del model
del trainer
clear_memory()

In [25]:
# Reload model in FP16 and merge with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, "gemma-2b-it-samsum")
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Login to Hugging Face hub and push model and tokenizer
# huggingface_hub.login(token=hf_token)
# tokenizer.push_to_hub(REPO_NAME, use_temp_dir=False)
# model.push_to_hub(REPO_NAME, use_temp_dir=False)

In [ ]:
# Zip up the model folder
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
# !zip -r /content/gemma-2b-it-samsum.zip /content/gemma-2b-it-samsum

In [ ]:
# Copy model to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/gemma-2b-it-samsum.zip /content/drive/MyDrive/gemma-2b-it-samsum.zip